In [2]:
!pip install gdown -U
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00


In [3]:
import os
os.environ["WANDB_DISABLED"]="true"

In [4]:
!mkdir -p results_docs_bert

In [5]:
# cargar dataset de google drive
!gdown '1pl9hGX3_4lMJvl4thbwkEXRCTi_dH1au'

Downloading...
From: https://drive.google.com/uc?id=1pl9hGX3_4lMJvl4thbwkEXRCTi_dH1au
To: /kaggle/working/ReWoRC_Dataset_req.csv
100%|█████████████████████████████████████████| 437k/437k [00:00<00:00, 104MB/s]


Aqui empiezo el código igual al de colab

In [6]:
from datasets import load_dataset

In [7]:
FILE_REQ='/kaggle/working/ReWoRC_Dataset_req.csv'
data_req = load_dataset('csv', data_files=FILE_REQ, sep=';')
print(data_req['train']['NFR'][21])

def create_labels(row):
  return {'labels': 'F' if row['NFR'] == 'Funcional' else 'NF', 'id_doc':row['Documento']}

data_req = data_req.map(create_labels)
data_req = data_req.class_encode_column('labels')
data_req = data_req.class_encode_column('id_doc')
print(set(data_req['train']['labels']))
data_req = data_req['train']
data_req.features

Generating train split: 0 examples [00:00, ? examples/s]

No funcional


Map:   0%|          | 0/2391 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/2391 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/2391 [00:00<?, ? examples/s]

{0, 1}


{'Num': Value(dtype='int64', id=None),
 'Referencia': Value(dtype='string', id=None),
 'Requisito': Value(dtype='string', id=None),
 'Documento': Value(dtype='string', id=None),
 'NFR': Value(dtype='string', id=None),
 'labels': ClassLabel(names=['F', 'NF'], id=None),
 'id_doc': ClassLabel(names=['0000 - gamma j.pdf', '0000 - inventory.pdf', '1998 - themas.pdf', '1999 - multi-mahjong.html', '2001 - beyond.pdf', '2001 - ctc network.pdf', '2001 - esa.pdf', '2001 - hats.pdf', '2001 - libra.doc', '2001 - space fractions.pdf', '2002 - evla back.pdf', '2002 - evla corr.pdf', '2003 - agentmom.pdf', '2003 - pnnl.pdf', '2003 - qheadache.pdf', '2003 - tachonet.pdf', '2004 - colorcast.pdf', '2004 - ijis.doc', '2004 - phillips.doc', '2004 - rlcs.pdf', '2004 - sprat.pdf', '2005 - clarus low.pdf', '2005 - grid 3D.pdf', '2005 - microcare.pdf', '2005 - nenios.html', '2005 - phin.pdf', '2005 - pontis.pdf', '2005 - triangle.pdf', '2005 - znix.pdf', '2006 - stewards.pdf', '2007 - e-store.pdf', '2007 - ei

In [22]:
from transformers import AutoTokenizer
#BERT: model_name = 'google-bert/bert-base-cased'
#DEBERTA: model_name = 'microsoft/deberta-v3-base'
#ROBERTA: model_name = 'FacebookAI/roberta-base'
model_name = 'google-bert/bert-base-cased' # modificar con el modelo que queramos cada vez

In [23]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [24]:
def encode_text(row, tokenizer):
  row_encode = tokenizer(row['Requisito'], truncation=True)
  return row_encode


encode_data = data_req.map(encode_text, fn_kwargs={'tokenizer':tokenizer})
encode_data

from sklearn.model_selection import KFold
import numpy as np

folds = KFold(n_splits=10, shuffle=True)

splits = folds.split(np.zeros(50))
test_sets = []
train_val_sets = []
i=1
for train_val_idxs, test_idxs in splits:
  print (i)
  print(test_idxs)
  print('*******')
  data_test = encode_data.filter(lambda r : r['id_doc'] in test_idxs)
  data_train = encode_data.filter(lambda r : r['id_doc'] in train_val_idxs)
  data_train_val = data_train.train_test_split(train_size=0.9, stratify_by_column='labels')
  print(data_test)
  print(data_train)
  print(data_test.num_rows + data_train.num_rows)
  print(encode_data.num_rows == (data_test.num_rows + data_train.num_rows))
  print("***")


  test_sets.append(data_test)
  train_val_sets.append(data_train_val)
  i = i + 1

print(test_sets[0])
print(train_val_sets[0]['test'])

Map:   0%|          | 0/2391 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


1
[13 17 30 39 45]
*******


Filter:   0%|          | 0/2391 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2391 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/2391 [00:00<?, ? examples/s]

Dataset({
    features: ['Num', 'Referencia', 'Requisito', 'Documento', 'NFR', 'labels', 'id_doc', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 183
})
Dataset({
    features: ['Num', 'Referencia', 'Requisito', 'Documento', 'NFR', 'labels', 'id_doc', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2208
})
2391
True
***
10
[ 7 14 28 38 42]
*******


Filter:   0%|          | 0/2391 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2391 [00:00<?, ? examples/s]

Dataset({
    features: ['Num', 'Referencia', 'Requisito', 'Documento', 'NFR', 'labels', 'id_doc', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 288
})
Dataset({
    features: ['Num', 'Referencia', 'Requisito', 'Documento', 'NFR', 'labels', 'id_doc', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2103
})
2391
True
***
Dataset({
    features: ['Num', 'Referencia', 'Requisito', 'Documento', 'NFR', 'labels', 'id_doc', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 249
})
Dataset({
    features: ['Num', 'Referencia', 'Requisito', 'Documento', 'NFR', 'labels', 'id_doc', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 215
})


In [25]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
import evaluate
import numpy as np

In [27]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return metric.compute(predictions=predictions, references=labels)

# load metric
metric_name = 'f1'
metric = evaluate.load(metric_name)

In [28]:
import glob
from transformers import TrainingArguments, Trainer
from sklearn.metrics import classification_report

lr_values =[2e-5, 5e-5]

for j in range(len(train_val_sets)):
    best_lr = -1
    best_f1 = -1

    # para escribir los resultados
    with open(f'results_docs_bert/res_docs_bert_{j}.txt', 'w') as f:
    # NOTA: si no va, deshacer el tab extra del siguiente bucle for
      for i, lr in enumerate(lr_values):
        print("================================================")
        print("j = " + str(j) + " // lr = " + str(lr))
        print("================================================")
      # argumentos para el entrenamiento
        training_args = TrainingArguments(
            #output_dir="my_checkpoint"+str(j),
            output_dir=f"my_checkpoint_{i}",
            overwrite_output_dir = True,
            num_train_epochs=10,
            #para probar tanto promise como nuestro coger 10 epochs
            evaluation_strategy="epoch",
            save_strategy="epoch",
            save_total_limit = 1,
            load_best_model_at_end=True,
            logging_strategy='epoch',
            optim="adamw_torch",
            per_device_train_batch_size=32,
            #lo dejaremos en 32
            #per_device_eval_batch_size=32,
            learning_rate=lr,
            #probar tanto 2e-5 como 5e-5
            weight_decay=0.01,
        )

        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
        # declarar el "entrenador"
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_val_sets[j]['train'],
            eval_dataset=train_val_sets[j]['test'],
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )

        # realizar el entrenamiento
        trainer.train()

        out_pred = trainer.predict(data_train_val['test'])
        if out_pred.metrics['test_f1'] > best_f1:
          best_f1 = out_pred.metrics['test_f1']
          best_lr = i
      # cargar el mejor modelo y resultados para test
      best_dir = glob.glob(f"my_checkpoint_{best_lr}/checkpoint*")
      print(best_dir)
      model_best = AutoModelForSequenceClassification.from_pretrained(best_dir[0])
      trainer_best = Trainer(
            model=model_best,
            args=None,
            train_dataset=None,
            eval_dataset=None,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )

      test_data = test_sets[j]
      outputs_pred = trainer_best.predict(test_dataset=test_data)

      # model predictions
      predictions = np.argmax(outputs_pred.predictions, axis=1)

      #d_res = classification_report(test_data['labels'], predictions, digits=3, return_dict = True)
      d_res = classification_report(test_data['labels'], predictions, digits=3, output_dict=True)
      print(d_res)
      f.write("================== Best lr ================\n")
      f.write(str(best_lr))
      f.write("\n")
      f.write( "================== Resultados ================\n")
      f.write(str(d_res))

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


j = 0 // lr = 2e-05


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,0.417800,0.234118,0.776119
2,0.197700,0.130564,0.862069
3,0.133600,0.132788,0.830769
4,0.098000,0.179714,0.763636
5,0.051100,0.186520,0.825397
6,0.030300,0.221070,0.805970
7,0.018400,0.224874,0.843750
8,0.009000,0.219526,0.857143
9,0.009600,0.255440,0.843750
10,0.008200,0.244914,0.857143


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


...

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,0.372600,0.161894,0.816901
2,0.180200,0.110980,0.857143
3,0.132400,0.091577,0.906250
4,0.060000,0.085825,0.898551
5,0.041000,0.095063,0.927536
6,0.015900,0.149478,0.885246
7,0.015400,0.240069,0.866667
8,0.010900,0.250803,0.866667
9,0.009000,0.231970,0.866667
10,0.008700,0.166327,0.885246


['my_checkpoint_1/checkpoint-240']


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


{'0': {'precision': 0.9705882352941176, 'recall': 0.7983870967741935, 'f1-score': 0.8761061946902654, 'support': 248}, '1': {'precision': 0.40476190476190477, 'recall': 0.85, 'f1-score': 0.5483870967741935, 'support': 40}, 'accuracy': 0.8055555555555556, 'macro avg': {'precision': 0.6876750700280112, 'recall': 0.8241935483870968, 'f1-score': 0.7122466457322294, 'support': 288}, 'weighted avg': {'precision': 0.8920012449424214, 'recall': 0.8055555555555556, 'f1-score': 0.8305896533130332, 'support': 288}}


Aquí acaba el código de colab. Falta hacer el zip para poder descargar los resultados (y opcionalmente usar el bot para que me avise del final de la ejecución)

In [29]:
import requests

# Definir la URL base y los parámetros
url = "https://api.callmebot.com/whatsapp.php"
params = {
    "phone": "+34653508040",
    "apikey": "1512070",
    "text": "🤖¡Ejecución terminada! 🤖\nVe a echarle un vistazo ☝🤓"
}

# Enviar la solicitud GET
response = requests.get(url, params=params)

# Imprimir el estado de la respuesta
print(response.status_code)
print(response.text)

200
<p>Message to: +34653508040<p>Text to send: 🤖¡Ejecución terminada! 🤖%0AVe a echarle un vistazo ☝🤓<p><b>Message queued.</b> You will receive it in a few seconds.


In [31]:
!zip -r results_docs_bert.zip results_docs_bert
from IPython.display import FileLink 
FileLink(r'results_docs_bert.zip')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


updating: results_docs_deberta/ (stored 0%)
updating: results_docs_deberta/res_docs_deberta_8.txt (deflated 57%)
updating: results_docs_deberta/res_docs_deberta_7.txt (deflated 57%)
updating: results_docs_deberta/res_docs_deberta_9.txt (deflated 58%)
updating: results_docs_deberta/res_docs_deberta_3.txt (deflated 67%)
updating: results_docs_deberta/res_docs_deberta_2.txt (deflated 56%)
updating: results_docs_deberta/res_docs_deberta_1.txt (deflated 57%)
updating: results_docs_deberta/res_docs_deberta_5.txt (deflated 58%)
updating: results_docs_deberta/res_docs_deberta_0.txt (deflated 57%)
updating: results_docs_deberta/res_docs_deberta_6.txt (deflated 57%)
updating: results_docs_deberta/res_docs_deberta_4.txt (deflated 58%)


/kaggle/working/results_docs_deberta.zip